In [1]:
from v2_rebalance_dashboard.constants import ROOT_PRICE_ORACLE_ABI, ROOT_PRICE_ORACLE, eth_client
from v2_rebalance_dashboard.get_events import fetch_events

flash_borrow_solver = "0x2C26808b567BA224652f4eB20D45df4bccC29470"
root_price_oracle_contract = eth_client.eth.contract(ROOT_PRICE_ORACLE, abi=ROOT_PRICE_ORACLE_ABI)
tokens = fetch_events(root_price_oracle_contract.events.TokenRegistered)["token"].values

2024-09-11 14:50:26.900 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


,date,block,break_even_days,swapCost,apr_delta,out_compositeReturn,in_compositeReturn,predicted_increase_after_swap_cost,predicted_gain_during_swap_cost_off_set_period,inEthValue,outEthValue,out_destination,in_destination,offset_period,slippage,gasCostInETH
0,2024-07-17 16:22:11,20327410,12.075341,0.213994,10.819212,0.000000,10.819212,0.849300,1.063294,59.786006,60.000000,balETH idle,Balancer rsETH / ETHx,60,0.003567,0
1,2024-08-07 18:16:35,20478430,2.677286,0.016958,11.332331,0.000000,11.332331,0.363085,0.380043,20.401195,20.418153,balETH idle,Balancer rsETH-WETH Stable Pool,60,0.000831,0.0258890225734398
2,2024-08-08 13:38:59,20484219,51.506121,0.055676,0.668006,9.688980,10.356986,0.009181,0.064857,59.871058,59.926734,Balancer rsETH / ETHx,Balancer rsETH-WETH Stable Pool,60,0.000929,0.072464535737705545
3,2024-08-17 01:39:59,20545121,43.717981,0.112094,1.174135,6.314949,7.489085,0.041747,0.153841,80.309649,80.421742,Balancer rsETH-WETH Stable Pool,Balancer rsETH / ETHx,60,0.001394,0.017798404538810774
4,2024-08-23 16:39:47,20592551,21.103624,0.074363,1.604792,8.214074,9.818866,0.137059,0.211421,80.524713,80.599076,Balancer rsETH / ETHx,Balancer ETHx/wstETH,60,0.000923,0.025683613073514642


In [24]:
from multicall import Call
import pandas as pd
from v2_rebalance_dashboard.get_state_by_block import (
    safe_normalize_with_bool_success,
    identity_with_bool_success,
    sync_get_raw_state_by_block_one_block,
    to_str_with_bool_success,
    sync_safe_get_raw_state_by_block,
    build_get_address_eth_balance_call,
)
from v2_rebalance_dashboard.get_rebalance_events_summary import _fetch_rebalance_events_df

clean_rebalance_df = _fetch_rebalance_events_df()


def getPriceInEth_call(name: str, token_address: str) -> Call:
    return Call(
        ROOT_PRICE_ORACLE,
        ["getPriceInEth(address)(uint256)", token_address],
        [(name, safe_normalize_with_bool_success)],
    )


def _build_value_held_by_solver(balance_of_calls, price_calls, blocks):
    blocks = [int(b) for b in blocks]
    balance_of_df = sync_safe_get_raw_state_by_block(balance_of_calls, blocks)
    price_df = sync_safe_get_raw_state_by_block(price_calls, blocks).fillna(0)  # just usdc is 0s , can ignore
    price_df["ETH"] = 1.0
    price_df = price_df[balance_of_df.columns]
    eth_value_held_by_flash_solver_df = price_df * balance_of_df
    eth_value_held_by_flash_solver_df["total_eth_value"] = eth_value_held_by_flash_solver_df.sum(axis=1)
    return eth_value_held_by_flash_solver_df


def add_solver_profit_cols(tokens: list[str], clean_rebalance_df: pd.DataFrame) -> list[Call]:

    symbol_calls = [Call(t, ["symbol()(string)"], [(t, to_str_with_bool_success)]) for t in tokens]
    address_to_symbol = sync_get_raw_state_by_block_one_block(symbol_calls, 20651330)
    balance_of_calls = [build_get_address_eth_balance_call("ETH", flash_borrow_solver)]

    price_calls = []
    for token_addr, symbol in address_to_symbol.items():
        if symbol is not None:
            balance_of_calls.append(
                Call(
                    token_addr,
                    ["balanceOf(address)(uint256)", flash_borrow_solver],
                    [(symbol, safe_normalize_with_bool_success)],
                )
            )
            price_calls.append(getPriceInEth_call(symbol, token_addr))

    value_before_df = _build_value_held_by_solver(balance_of_calls, price_calls, clean_rebalance_df["block"] - 1)
    value_after_df = _build_value_held_by_solver(balance_of_calls, price_calls, clean_rebalance_df["block"])

    clean_rebalance_df["before_rebalance_eth_value_of_solver"] = value_before_df["total_eth_value"].values
    clean_rebalance_df["after_rebalance_eth_value_of_solver"] = value_after_df["total_eth_value"].values
    clean_rebalance_df["solver_profit"] = (
        clean_rebalance_df["after_rebalance_eth_value_of_solver"]
        - clean_rebalance_df["before_rebalance_eth_value_of_solver"]
    )
    return clean_rebalance_df


clean_rebalance_df = add_solver_profit_cols(tokens, clean_rebalance_df)
clean_rebalance_df

,date,block,break_even_days,swapCost,apr_delta,out_compositeReturn,in_compositeReturn,predicted_increase_after_swap_cost,predicted_gain_during_swap_cost_off_set_period,inEthValue,outEthValue,out_destination,in_destination,offset_period,slippage,gasCostInETH,before_rebalance_eth_value_of_solver,after_rebalance_eth_value_of_solver,solver_profit
0,2024-07-17 16:22:11,20327410,12.075341,0.213994,10.819212,0.000000,10.819212,0.849300,1.063294,59.786006,60.000000,balETH idle,Balancer rsETH / ETHx,60,0.003567,0,0.000000,0.003000,0.003000
1,2024-08-07 18:16:35,20478430,2.677286,0.016958,11.332331,0.000000,11.332331,0.363085,0.380043,20.401195,20.418153,balETH idle,Balancer rsETH-WETH Stable Pool,60,0.000831,0.0258890225734398,0.012501,0.008902,-0.003599
2,2024-08-08 13:38:59,20484219,51.506121,0.055676,0.668006,9.688980,10.356986,0.009181,0.064857,59.871058,59.926734,Balancer rsETH / ETHx,Balancer rsETH-WETH Stable Pool,60,0.000929,0.072464535737705545,0.008896,0.018825,0.009928
3,2024-08-17 01:39:59,20545121,43.717981,0.112094,1.174135,6.314949,7.489085,0.041747,0.153841,80.309649,80.421742,Balancer rsETH-WETH Stable Pool,Balancer rsETH / ETHx,60,0.001394,0.017798404538810774,0.010272,0.010998,0.000725
4,2024-08-23 16:39:47,20592551,21.103624,0.074363,1.604792,8.214074,9.818866,0.137059,0.211421,80.524713,80.599076,Balancer rsETH / ETHx,Balancer ETHx/wstETH,60,0.000923,0.025683613073514642,0.011001,0.019859,0.008858


In [23]:
clean_rebalance_df

In [22]:
clean_rebalance_df["block"] - 1

TypeError: 'NoneType' object is not subscriptable

In [8]:
value_after_df["total_eth_value"].values - value_before_df["total_eth_value"].values

array([ 0.003     , -0.00359887,  0.00992832,  0.00072544,  0.00885845])

In [ ]:
value_after_df

In [ ]:
eth_value_held_by_flash_solver_df

In [ ]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

px.line(eth_value_held_by_flash_solver_df[eth_value_held_by_flash_solver_df.columns[:-1]])

In [ ]:
px.bar(eth_value_held_by_flash_solver_df, y=eth_value_held_by_flash_solver_df.columns[:-1])

In [ ]:
# at the end of each rebalance I want to know the quantity of each token they have and the price



tokens =

from multicall import Call




In [ ]:
from v2_rebalance_dashboard.solver_diagnostics import load_solver_df

solver_df = load_solver_df()
solver_df

In [ ]:
solver_df["solverAddress"].value_counts()